In [1]:
from selenium.webdriver import Edge
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
import re

In [3]:
import pandas as pd

In [4]:
pd.options.display.max_columns = 40

In [85]:
start_uri = "https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/2021/schedule/#fromDate=2021-05-25"
weeks = 4
match_id = 1

In [86]:
stats_dataframes = pd.DataFrame()

In [87]:
options = Options()
service = Service('msedgedriver.exe')
browser = Edge(options=options, service=service)

In [88]:
browser.get(start_uri)

In [89]:
# click to close cookies request
browser.execute_script(
    'arguments[0].click();',
    browser.find_element('xpath', '//*[@id="onetrust-close-btn-container"]/button')
)

In [90]:
# click to open gender dropdown
browser.execute_script(
    'arguments[0].click();',
    browser.find_element(By.XPATH, '/html/body/div[1]/main/section/div[1]/div/div[3]/div[2]/div[1]/div/div[3]/div')
)

In [91]:
# select 'woman' in dropdown
browser.execute_script(
    'arguments[0].click();',
    browser.find_element('xpath', '/html/body/div[1]/main/section/div[1]/div/div[3]/div[2]/div[1]/div/div[3]/ul/li[2]')
)

In [92]:
# get all cards of matches
matches_card = browser.find_elements(
    By.CSS_SELECTOR, 
    'div.vbw-gs2-match-data-card'
)

In [ ]:
for match_card in matches_card:
    match_url = match_card.find_element('css selector', 'a').get_attribute('href')
    browser.get(match_url)
    
    browser.implicitly_wait(5)

    table = browser.find_element('xpath', '/html/body/div[1]/main/div/section/div/div[1]/div[1]/div[2]/div/table')
    stats_table = pd.read_html(table.get_attribute('outerHTML'))[0]
    stats_table.to_excel(f'')

In [93]:
match_url = matches_card[0].find_element('css selector', 'a').get_attribute('href')
browser.get(match_url)

### repete a partir daqui

In [110]:
pool, phase, matchN = browser.find_element('xpath', '//*[@id="main-content"]/section/div/div/div[2]/div/a[1]/div[2]/div[1]').text.split(' - ')
pool = re.sub('\D+', '', pool)
matchN = re.sub('\D+', '', matchN)
pool, phase, matchN

('1', 'Preliminary Phase', '2')

In [111]:
arena = re.sub(
    r'\n', 
    '-', 
    browser.find_element('xpath', '//*[@id="main-content"]/section/div/div/div[2]/div/a[1]/div[2]/div[3]').text
)
arena

'Rimini-Italy'

In [112]:
home_team = browser.find_element('xpath', '//*[@id="main-content"]/section/div/div/div[2]/div/a[2]/div[1]/div[1]/div[2]').get_attribute('innerHTML')
away_team = browser.find_element('xpath', '//*[@id="main-content"]/section/div/div/div[2]/div/a[2]/div[1]/div[3]/div[2]').get_attribute('innerHTML')
home_team, away_team

('Türkiye', 'Serbia')

In [113]:
home_res, away_res = browser.find_element('xpath', '//*[@id="main-content"]/section/div/div/div[2]/div/a[2]/div[1]/div[2]/div[1]').text.split('\n:\n')
home_res, away_res

('3', '2')

In [114]:
table = browser.find_element('xpath', '/html/body/div[1]/main/div/section/div/div[1]/div[1]/div[2]/div/table')

In [115]:
stats_table = pd.read_html(table.get_attribute('outerHTML'))[0]

C:\Users\Leandro\AppData\Local\Temp\ipykernel_13708\2984983427.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats_table = pd.read_html(table.get_attribute('outerHTML'))[0]


In [116]:
stats_table

,TürkiyeTUR,TürkiyeTUR.1,Match StatsStats,SerbiaSRB,SerbiaSRB.1
0,NaN,57,Attack,56,NaN
1,NaN,13,Block,18,NaN
2,NaN,6,Serve,9,NaN
3,NaN,30,Opponent Error,25,NaN
4,NaN,106,Total,108,NaN
5,Match Skills,Match Skills,Match Skills,Match Skills,Match Skills
6,NaN,94,Dig,87,NaN
7,NaN,94,Reception,99,NaN
8,NaN,157,Set,164,NaN
9,Boz(13),17,Best Scorers,22,Caric(3)


In [117]:
stats_colsA = (stats_table['Match StatsStats'] + ' A').tolist()
stats_colsB = (stats_table['Match StatsStats'] + ' B').tolist()
stats_colsA[-1] = stats_colsA[-1]+'2'
stats_colsB[-1] =  stats_colsB[-1]+'2'

stats_colsA, stats_colsB

(['Attack A',
  'Block A',
  'Serve A',
  'Opponent Error A',
  'Total A',
  'Match Skills A',
  'Dig A',
  'Reception A',
  'Set A',
  'Best Scorers A',
  'Best Scorers A2'],
 ['Attack B',
  'Block B',
  'Serve B',
  'Opponent Error B',
  'Total B',
  'Match Skills B',
  'Dig B',
  'Reception B',
  'Set B',
  'Best Scorers B',
  'Best Scorers B2'])

In [118]:
stats_df_clean = pd.DataFrame(columns=[*stats_colsA, *stats_colsB])

In [103]:
def set_row(columns, values):    
    for col, value in zip(columns, values):
        lst_idx = stats_df_clean.last_valid_index()
        lst_idx = lst_idx+1 if lst_idx is not None else 0
        stats_df_clean.loc[match_id, col] = value
    

In [119]:
stats_df_clean.loc[match_id, 'match_id'] = match_id
stats_df_clean.loc[match_id, 'home'] = home_team
stats_df_clean.loc[match_id, 'away'] = away_team
stats_df_clean.loc[match_id, 'home_res'] = home_res
stats_df_clean.loc[match_id, 'away_res'] = away_res

stats_df_clean.loc[match_id, 'pool'] = pool
stats_df_clean.loc[match_id, 'phase'] = phase
stats_df_clean.loc[match_id, 'matchN'] = matchN
stats_df_clean.loc[match_id, 'arena'] = arena
set_row(stats_colsA, stats_table.iloc[:,1])
set_row(stats_colsB, stats_table.iloc[:,3])

In [120]:
stats_df_clean

,Attack A,Block A,Serve A,Opponent Error A,Total A,Match Skills A,Dig A,Reception A,Set A,Best Scorers A,Best Scorers A2,Attack B,Block B,Serve B,Opponent Error B,Total B,Match Skills B,Dig B,Reception B,Set B,Best Scorers B,Best Scorers B2,match_id,home,away,home_res,away_res,pool,phase,matchN,arena
2,57,13,6,30,106,Match Skills,94,94,157,17,16,56,18,9,25,108,Match Skills,87,99,164,22,17,2.0,Türkiye,Serbia,3,2,1,Preliminary Phase,2,Rimini-Italy


In [121]:
stats_dataframes = pd.concat([stats_dataframes, stats_df_clean], axis=0)

In [122]:
stats_dataframes

,Attack A,Block A,Serve A,Opponent Error A,Total A,Match Skills A,Dig A,Reception A,Set A,Best Scorers A,Best Scorers A2,Attack B,Block B,Serve B,Opponent Error B,Total B,Match Skills B,Dig B,Reception B,Set B,Best Scorers B,Best Scorers B2,match_id,home,away,home_res,away_res,pool,phase,matchN,arena
1,47,6,5,17,75,Match Skills,57,49,102,13,12,35,2,2,19,58,Match Skills,26,64,97,13,11,1.0,Netherlands,Belgium,3,0,1,Preliminary Phase,1,Rimini-Italy
2,57,13,6,30,106,Match Skills,94,94,157,17,16,56,18,9,25,108,Match Skills,87,99,164,22,17,2.0,Türkiye,Serbia,3,2,1,Preliminary Phase,2,Rimini-Italy


In [123]:
match_id += 1

#### vai para proxima partida

In [124]:
upcomming = browser.find_element('xpath', '//*[@id="main-content"]/div/section/div/div[2]/section/div[2]/div[1]/div/div[1]/a').get_attribute('innerHTML')
if upcomming != 'Match Centre':
    # break
    print('has no next')
else:
    print('has next')
    browser.execute_script(
        'arguments[0].click();',
        browser.find_element('xpath', '//*[@id="main-content"]/div/section/div/div[2]/section/div[2]/div[1]/div/div[1]/a')
    )

has next


### Parte de dados de jogadoras

In [34]:
# go to 'box score'
browser.execute_script(
    'arguments[0].click();', 
    browser.find_element('xpath', '/html/body/div[1]/main/div/section/div/div[1]/div[1]/div[1]/ul/li[2]/a')
)

In [35]:
box_tables = browser.find_elements('xpath', '//*[@id="main-content"]/div/section/div/div[1]/div[1]/div[3]/div/div[2]/div/div[2]/div/div/table')

In [37]:
box_dfs = []
for t in box_tables:
    box_dfs.append(
        pd.read_html(t.get_attribute('outerHTML'))[0]
    )

C:\Users\Leandro\AppData\Local\Temp\ipykernel_13708\4268681772.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(t.get_attribute('outerHTML'))[0]
C:\Users\Leandro\AppData\Local\Temp\ipykernel_13708\4268681772.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(t.get_attribute('outerHTML'))[0]
C:\Users\Leandro\AppData\Local\Temp\ipykernel_13708\4268681772.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(t.get_attribute('outerHTML'))[0]
C:\Users\Leandro\AppData\Local\Temp\ipykernel_13708\4268681772.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will

In [38]:
dfs_A = box_dfs[:28]
dfs_B = box_dfs[28:]

In [129]:
dfs_A[0]

,Player NoNo,Player NamePlayer,PositionPos.,Total ABSABS,Attack PointsA Pts,Block PointsB Pts,Serve PointsS Pts,ErrorsErr,Efficiency %Eff
0,26,Dambrink,O,13,10,2,1,NaN,28.57
1,11,Buijs (C),OH,12,12,0,0,NaN,8.89
2,16,Baijens,MB,10,9,0,1,NaN,0.00
3,19,Daalderop,OH,10,7,2,1,NaN,6.82
4,23,Timmerman,MB,10,7,1,2,NaN,3.03
5,12,Bongaerts,S,3,2,1,0,NaN,5.26
6,1,Knip,L,0,0,0,0,NaN,NaN
7,2,Savelkoel,OH,0,0,0,0,NaN,NaN
8,8,Korevaar,MB,0,0,0,0,NaN,0.00
9,9,Schoot,L,0,0,0,0,NaN,NaN


In [130]:
dfs_B[0]

,Player NoNo,Player NamePlayer,PositionPos.,Total ABSABS,Attack PointsA Pts,Block PointsB Pts,Serve PointsS Pts,ErrorsErr,Efficiency %Eff
0,3,Herbots,OH,13,13,0,0,NaN,11.76
1,13,Janssens,MB,11,10,1,0,NaN,8.33
2,5,Guilliams,OH,5,5,0,0,NaN,3.57
3,7,Van Gestel,OH,5,4,0,1,NaN,-8.57
4,17,Van de Vyver I. (C),S,3,1,1,1,NaN,0.00
5,10,Sobolska,MB,2,2,0,0,NaN,-28.57
6,2,Van Sas,S,0,0,0,0,NaN,NaN
7,6,Gilson,OH,0,0,0,0,NaN,NaN
8,9,Demeyer,OH,0,0,0,0,NaN,NaN
9,14,De Valkeneer,OH,0,0,0,0,NaN,NaN


In [ ]:
['all_sets', 'set1', 'set2', 'set3']
[
    'SCORING',
    'ATTACK',
    'BLOCK',
    'SERVE',
    'RECEPTION',
    'DIG',
    'SET',
]


In [40]:
dfs_B[-1]

,Player NoNo,Player NamePlayer,PositionPos.,PointPt,ErrorsErr,AttemptsAtt,TotalTot.,Efficiency %Eff
0,17,Van de Vyver I. (C),S,8,0,22,30,26.67
1,2,Van Sas,S,0,0,0,0,NaN
2,3,Herbots,OH,0,0,0,0,NaN
3,5,Guilliams,OH,0,0,0,0,NaN
4,6,Gilson,OH,0,0,0,0,NaN
5,7,Van Gestel,OH,0,0,0,0,NaN
6,9,Demeyer,OH,0,0,0,0,NaN
7,10,Sobolska,MB,0,0,0,0,NaN
8,13,Janssens,MB,0,0,0,0,NaN
9,14,De Valkeneer,OH,0,0,0,0,NaN


In [ ]:
len(box_dfs)

In [125]:
browser.quit()